In [1]:
import torch

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

from trl import setup_chat_format

C:\Users\USER_ELISEY\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
path_to_source_model = "C:\\Users\\USER_ELISEY\\qwen"
path_to_finetuned_model = "C:\\Users\\USER_ELISEY\\hakaton\\russia_mini_chad\\checkpoint-100"

# Compare models

In [2]:
# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

# Load model
casual_model = AutoModelForCausalLM.from_pretrained(
    path_to_source_model,
    quantization_config=bnb_config,
#     device_map="auto",
    attn_implementation="eager"
)

tokenizer = tokenizer = AutoTokenizer.from_pretrained(path_to_source_model)
tokenizer.padding_side = 'right'
tokenizer.padding_token = '<|pad_token|>'

`low_cpu_mem_usage` was None, now set to True since model is quantized.


In [3]:
casual_model, tokenizer = setup_chat_format(casual_model, tokenizer)

In [4]:
model = AutoModelForCausalLM.from_pretrained(
    path_to_finetuned_model,
    quantization_config=bnb_config,
#     device_map="auto",
    attn_implementation="eager"
)

tokenizer = tokenizer = AutoTokenizer.from_pretrained(path_to_finetuned_model)
tokenizer.padding_side = 'right'
tokenizer.padding_token = '<|pad_token|>'

`low_cpu_mem_usage` was None, now set to True since model is quantized.


In [5]:
model, tokenizer = setup_chat_format(model, tokenizer)

## Get answers

In [6]:
def generate_answer(model, prompt):
    chat = [
        { "role": "user", "content": prompt },
    ]
    prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
    outputs = model.generate(input_ids=inputs.to(model.device), max_new_tokens=150)

    return(tokenizer.decode(outputs[0]))

# Comprasion

In [7]:
q1 = "В каком году Александр Невский поехал в орду?"
q2 = "Как звали сына Сталина?"
q3 = "Кто такой Брежнев?"

In [8]:
print(generate_answer(model, q1))

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


<|im_start|>user
В каком году Александр Невский поехал в орду?<|im_end|>
<|im_start|>assistant
Александр Невский поехал в орду в 1887 году.<|im_end|>


In [9]:
print(generate_answer(model, q2))

<|im_start|>user
Как звали сына Сталина?<|im_end|>
<|im_start|>assistant
Сын Сталина — это имя, которое было названо в 1930 году, когда впервые в истории СССР было официально названо имя Сталина. В 1930 году в СССР было официально названо имя Сталина, но в 1931 году было официально названо имя Сталина.<|im_end|>


In [10]:
print(generate_answer(model, q3))

<|im_start|>user
Кто такой Брежнев?<|im_end|>
<|im_start|>assistant
Брежнев — советский и российский политический деятель, член ЦК КПСС, председатель ЦК КПСС СССР, председатель ЦК КПСС СССР, председатель ЦК КПСС СССР, председатель ЦК КПСС СССР, председатель ЦК КПСС СССР, председатель ЦК КПСС СССР, председатель ЦК КПСС СССР, председатель ЦК КПСС СССР, председ


In [11]:
# # Free gpu memory
# import numba
# numba.cuda.close()

In [12]:
print(generate_answer(casual_model, q1))

<|im_start|>user
В каком году Александр Невский поехал в орду?<|im_end|>
<|im_start|>assistant
根据文本内容，以下为文本中关于“ Александр Невский поехал в орду”的部分：

1. Александр Невский поехал в орду в 1918 году.
2. Александр Невский был в орде с 1918 года по 1920 году.
3. Александр Невский был в орде с 1918 года по 1920 году.
4. Александр Невский был в орде с 1918 года по 1920 году.
5. Александр Невский был в орде с 1918 года по 1920 году.
6. Александр


In [13]:
print(generate_answer(casual_model, q2))

<|im_start|>user
Как звали сына Сталина?<|im_end|>
<|im_start|>assistant
Based on the information provided in the text, the name of the son of Stalin is not mentioned. The text only mentions that Stalin was the leader of the Soviet Union and that he was born in 1903. There is no mention of his name or any other information about his son.<|im_end|>


In [14]:
print(generate_answer(casual_model, q3))

<|im_start|>user
Кто такой Брежнев?<|im_end|>
<|im_start|>assistant
The person who is the subject of the question is the former Soviet leader and Prime Minister of the Russian Federation, Boris Yeltsin.<|im_end|>
